### what are tools?
it allows the models to connect with the external functions.

### Usecases
1. Fetch extra data to add knowledge
2. take an action like a meeting
3. perform calculation
4. modify the UI. 

## Project - Airline AI Assistant

In [1]:
# imports
import os
from dotenv import load_dotenv
import openai
import gradio as gr
import json

In [2]:
## initialization
load_dotenv()
os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')
MODEL = 'gpt-4o-mini'
openai_client = openai.OpenAI()

In [3]:
## define the system prompt
system_prompt="""
You are a helpful assistant for an Airline called FlightAI.
Give short, concise, and  answers. No more than 1 senetence ever.
Always be accurate. If you are unsure about the answer. Be straightforward about that
"""

In [4]:
def chat(message, history):
    messages = [
        {"role":"system", "content": system_prompt}
    ]
    for user_message, assistant_message in history:
        messages.append({"role": "user", "content":user_message})
        messages.append({"role": "assistant", "content": assistant_message})
    messages.append({"role":"user", "content": message})

    result = openai_client.chat.completions.create(
        model = MODEL,
        messages = messages,
        stream = True
    )
    response = ""
    for chunk in result:
        response+=chunk.choices[0].delta.content or ""
        yield response

In [5]:
gr.ChatInterface(
    fn=chat,
    title="Airline Assistant"
).launch()

/Users/protikmostafa/miniconda3/envs/llms/lib/python3.11/site-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


In [6]:
## making a very generic function
ticket_prices = {
    "london": "$799",
    "paris": "$899",
    "tokyo": "$1400",
    "berlin": "$499"
}

def get_ticket_price(city):
    print(f"tool get_ticket_price called for city {city}")
    city = city.lower()
    return ticket_prices.get(city, "unknown")

In [7]:
get_ticket_price("berlin")

tool get_ticket_price called for city berlin


'$499'

In [8]:
## there is a particular dictionary structure that i srequired to describe our function as a tool

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a ticket to the destination city. Call this function whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to the city'",
    "parameters":{
        "type": "object",
        "properties":{
            "city":{
                "type": "string",
                "description": "The city the customer is travelling to",
            },
        },
    },
    "required": ["destination_city"],
    "additionalProperties": False
}


## we need to include this in a list of tools
tools = [
    {"type":"function", "function": price_function}
]

In [9]:
def chat(message, history):
    messages = [
        {"role":"system", "content": system_prompt}
    ]
    # old system
    # for human, assistant in history:
    #     messages.append({"role": "user", "content": human})
    #     messages.append({"role": "assistant", "content": assistant})
    for msg in history:
        messages.append(msg)
    messages.append({"role":"user", "content": message})

    result = openai_client.chat.completions.create(
        model = MODEL,
        messages = messages,
        tools = tools
    )
    ## in the next bit, we get the response normally with the choices
    ## in here, we will see that OpenAI will stop and like "I dont have the answer so I want to stop and call tools"

    if result.choices[0].finish_reason=="tool_calls":
        ## message is when OpenAI is asking us to call a tool
        message = result.choices[0].message
        # print(f"OpenAI collecting message: {message}")
        result,city = handle_tool_call(message)
        ## to the messages list of dict, we need to append 2 more rows
        ## 1. the message that OpenAI is asking us to use the tool
        ## 2. the response that we get from the tool
        messages.append(message)
        messages.append(result)
        result = openai_client.chat.completions.create(
            model = MODEL,
            messages = messages
        )
    return result.choices[0].message.content

In [10]:
## now the missing piece from our previous cell: handle_tools_call

def handle_tool_call(message):
    ## the message came back from OpenAI. we need to unpack to see which tool it did it want to call
    # print(f"message: {message}")
    tool_call = message.tool_calls[0]
    # print(f"tool_call = {tool_call}")
    arguments = json.loads(tool_call.function.arguments)
    # print(f"Arguments: {arguments}")
    city = arguments.get('city')
    price = get_ticket_price(city)
    result = {
        "role": "tool",
        "content": json.dumps({"city": city, "price": price}),
        "tool_call_id":message.tool_calls[0].id
    }
    return result, city

In [11]:
gr.ChatInterface(
    fn = chat,
    type = "messages"
).launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


tool get_ticket_price called for city Paris
tool get_ticket_price called for city Berlin
tool get_ticket_price called for city Bangladesh
